This Notebook will not be as verbose as the other ones as it is purely for the purpose to have the data ready for usage in the model later on.

No Steps within this notebook will be relevant for the report, but rather relevant to have the data accessible and usable in python.

In [20]:
import pandas as pd
from gluonts.dataset.pandas import PandasDataset
from gluonts.dataset import split
import xarray as ar
import cartopy
import scipy

In [30]:
#import all the data sets
climate = pd.read_csv("../../data/climate-daily.csv", header=0)

C:\Users\data-\AppData\Local\Temp\ipykernel_14072\1956721483.py:2: DtypeWarning: Columns (9,15,18,24,33) have mixed types. Specify dtype option on import or set low_memory=False.
  climate = pd.read_csv("../../data/climate-daily.csv", header=0)


In [31]:
for index in climate.dtypes.index:
    if climate.dtypes[index] == "object": 
        climate[index] = climate[index].astype('str')


In [45]:
climate["lon"] = climate["x"]
climate["lat"] = climate["y"]

In [47]:
climate.columns

Index(['x', 'y', 'LOCAL_MONTH', 'TOTAL_SNOW_FLAG', 'PROVINCE_CODE',
       'MAX_REL_HUMIDITY', 'HEATING_DEGREE_DAYS', 'STATION_NAME',
       'COOLING_DEGREE_DAYS_FLAG', 'MIN_REL_HUMIDITY_FLAG',
       'SNOW_ON_GROUND_FLAG', 'LOCAL_YEAR', 'MAX_TEMPERATURE',
       'SPEED_MAX_GUST', 'TOTAL_SNOW', 'CLIMATE_IDENTIFIER',
       'MAX_TEMPERATURE_FLAG', 'DIRECTION_MAX_GUST', 'DIRECTION_MAX_GUST_FLAG',
       'TOTAL_RAIN', 'COOLING_DEGREE_DAYS', 'MEAN_TEMPERATURE_FLAG',
       'MIN_TEMPERATURE', 'MIN_REL_HUMIDITY', 'MAX_REL_HUMIDITY_FLAG', 'ID',
       'TOTAL_RAIN_FLAG', 'MIN_TEMPERATURE_FLAG', 'TOTAL_PRECIPITATION_FLAG',
       'TOTAL_PRECIPITATION', 'SNOW_ON_GROUND', 'HEATING_DEGREE_DAYS_FLAG',
       'LOCAL_DATE', 'SPEED_MAX_GUST_FLAG', 'LOCAL_DAY', 'MEAN_TEMPERATURE',
       'lat', 'lon'],
      dtype='object')

In [53]:
columns = ["STATION_NAME","MAX_REL_HUMIDITY","TOTAL_PRECIPITATION","MEAN_TEMPERATURE","MIN_TEMPERATURE","MAX_TEMPERATURE","LOCAL_DATE","SNOW_ON_GROUND"]
relevant_climate = climate[columns]

In [56]:
#display(relevant_climate)
count = relevant_climate.groupby("STATION_NAME").count()

In [57]:
display(count)

,MAX_REL_HUMIDITY,TOTAL_PRECIPITATION,MEAN_TEMPERATURE,MIN_TEMPERATURE,MAX_TEMPERATURE,LOCAL_DATE,SNOW_ON_GROUND
STATION_NAME,,,,,,,
BACCARO POINT,0,0,605,607,605,608,376
BAS CARAQUET,0,10834,4354,4354,4354,10834,10255
CHURCHILL A,821,14308,14312,14313,14312,14313,12482
CHURCHILL FALLS,0,1386,972,1011,1129,1396,438
DEASE LAKE,0,14069,15109,15191,15330,15574,13339
EDMUNDSTON,0,8593,8486,8507,8496,8624,4324
FLIN FLON,0,18970,18999,19014,19042,19083,11501
GRAND FORKS,0,14251,14247,14252,14252,14259,9481
HIGH LEVEL A,633,659,659,659,659,669,659


In [58]:
weather_station_map ={}
north = ""
east = ""
south = ""
west = ""


for key in DGUID_map:
    province = DGUID_map[key]
    match province:
        case "Newfoundland and Labrador":
            north = "Nain"
            east = "Mary's Harbour"
            south = "Blanc-Sablon"
            west = "Churchill Falls"
            break
        case "Prince Edward Island":
            north = "St. Peters"
            east = "Charlottetown"
            south = "Maple Plains"
            west = "Summmerside"
            break
        case "Nova Scotia":
            north = "Sydney"
            east = "Malay Falls"
            south = "Baccaro Point"
            west = "Kentville"
            break
        case "New Brunswick":
            north = "Bas-Caraquet"
            east = "Moncton"
            south = "Saint John"
            west = "Edmundston"
            break
        case "Quebec":
            north = "Kuujjuaq"
            east = "Baie Comeau"
            south = "Maniwaki"
            west = "Kuujjuarapik"
            break
        case "Ontario":
            north = "Moosonee"
            east = "Ottawa"
            south = "Toronto"
            west = "Kenora"
            break
        case "Manitoba":
            north = "Churchill"
            east = "Gillam"
            south = "Winnipeg"
            west = "Flin Flon"
            break
        case "Saskatchewan":
            north = "Uranium City"
            east = "Yorkton"
            south = "Moose Jaw"
            west = "Meadow Lake"
            break
        case "Alberta":
            north = "High Level"
            west = "Jasper"
            south = "Lethbridge"
            east = "Lloydminster"
            break
        case "British Columbia":
            north = "Dease Lake"
            west = "Prince Rupert"
            south = "Grand Forks"
            east = "Prince George"
            break
    
    weather_station_map.update({province:{"north": north, 
                "east": east,
                "south": south, 
                "west": west}})

NameError: name 'DGUID_map' is not defined